In [ ]:
import gzip
import csv
import json
import pandas as pd
import numpy as np
#weather = pd.read_csv('FL2018.csv', header=0, error_bad_lines=False, low_memory=False)
#weather1 = pd.read_csv('CT2018.csv', header=0, parse_dates=False, low_memory=False)
#weather = weather.append(weather1)
#weather.reset_index(drop=True)
#weather.to_csv('weather.csv')
weather = pd.read_csv('weather.csv', header=0, error_bad_lines=False, low_memory=False)
weather['DATE'] = pd.to_datetime(weather['DATE'])
weather['DATE'] = weather.DATE.astype(str)
weather = weather.drop(weather.columns[[0]], axis=1)

In [ ]:
##Read in data
storms = pd.read_csv('StormEvents_details-ftp_v1.0_d2018_c20181017.csv.gz', compression='gzip', header=0)
#weather = pd.read_csv('1531535.csv', header=0, error_bad_lines=False, low_memory=False)
##Remove some unwanted columns
weather = weather.drop(weather.columns[[28,30,31,32,33,35,36,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89]], axis=1)
weather = weather.drop(weather.columns[[6,7,9]], axis=1)
##add location columns to weather
weather['COUNTY'] = np.nan
weather['ST'] = np.nan
weather.rename(columns={'STATION': 'WBAN'}, inplace=True)
weather['STATION_NAME'] = weather['STATION_NAME'].astype(str).str[:-5]
weather['WBAN'] = weather['WBAN'].str[5:].astype(np.int64)

In [ ]:
##Read in data for stations and their locations
locs = pd.read_fwf('emshr_lite_201810.txt', skiprows=[1], header=0)
locs = locs[locs.CTRY_NAME == 'UNITED STATES'].reset_index(drop=True)
col_list = ['WBAN', 'COUNTY', 'ST']
locs = locs[col_list]
locs = locs[pd.notnull(locs['COUNTY'])].reset_index(drop=True)
locs = locs[pd.notnull(locs['WBAN'])].reset_index(drop=True)
locs = locs[pd.notnull(locs['ST'])].reset_index(drop=True)
locs['WBAN'] = locs['WBAN'].astype(str).str[:-2].astype(np.int64)

In [ ]:
weather_data = pd.merge(weather, locs, on=['WBAN']) ###Merge station location data with weather based on WBAN

In [ ]:
storms.rename(columns={'CZ_NAME': 'COUNTY'}, inplace=True)

In [ ]:
event_type = storms['EVENT_TYPE'].unique() ##add event ID
event_type = pd.DataFrame(data=event_type, columns=['Weather']) ##Create table for the different types of storms

In [ ]:
event_type['EventID'] = event_type.index + 1

In [ ]:
storms = storms[storms.CZ_TYPE == 'C'].reset_index(drop=True) ##Only compare storms by county

In [ ]:
##ADD STATE COLUMN TO RESULT

In [ ]:
weather_data = weather_data[pd.notnull(weather_data['COUNTY_y'])].reset_index(drop=True)
weather_data = weather_data[pd.notnull(weather_data['ST_y'])].reset_index(drop=True)

In [ ]:
###REMOVE COUNTY X and ST X
weather_data = weather_data.drop(weather_data.columns[[29,30]], axis=1)
weather_data.rename(columns={'COUNTY_y': 'COUNTY'}, inplace=True)
weather_data.rename(columns={'ST_y': 'ST'}, inplace=True)

In [ ]:
### REFORMAT DATE AND TIME DATA

In [ ]:
weather_data['year'] = np.nan
weather_data['BEGIN_DAY'] = np.nan
weather_data['start_time'] = np.nan
weather_data['month'] = np.nan

In [ ]:
##split date into date and time, save time as time
f = lambda x: x["DATE"].rsplit(" ", 1)[1]
weather_data["start_time"] = weather_data.apply(f, axis=1)

In [ ]:
##split date to get day month year
f2 = lambda x: x["DATE"].rsplit(" ", 1)[0]
weather_data["BEGIN_DAY"] = weather_data.apply(f2, axis=1)
weather_data["year"] = weather_data.apply(f2, axis=1)
weather_data["month"] = weather_data.apply(f2, axis=1)

In [ ]:
f3 = lambda x: x["BEGIN_DAY"].split("-", 2)[2]
weather_data["BEGIN_DAY"] = weather_data.apply(f3, axis=1)

In [ ]:
f4 = lambda x: x["year"].split("-", 2)[0]
f5 = lambda x: x['month'].split("-",2)[1]
weather_data['year'] = weather_data.apply(f4, axis=1)
weather_data['month'] = weather_data.apply(f5, axis=1)


In [ ]:
##fill in zeroes at front
weather_data['month'] = weather_data['month'].apply(lambda x: x.zfill(2))

In [ ]:
##combine month and year
weather_data["BEGIN_YEARMONTH"] = weather_data["year"].map(str) + weather_data["month"].map(str)

In [ ]:
del weather_data['DATE']
del weather_data['year']
del weather_data['month']

In [ ]:
y = lambda x: x["start_time"].split(":", 2)[0]
z = lambda x: x["start_time"].split(":", 2)[1]
weather_data["start_time"] = weather_data.apply(y, axis=1) + weather_data.apply(z, axis=1) 

In [ ]:
##remove colon
#d = lambda x: x['start_time'].replace(':','')
#weather_data['start_time'] = weather_data.apply(d, axis=1)

In [ ]:
###drop monthly columns
states = pd.read_csv('states.csv', delimiter=',', header=0)
weather_data = pd.merge(weather_data, states, on=['ST'])
weather_data['STATE'] = weather_data['STATE'].str.upper()


In [ ]:
weather_data['end_time'] = np.nan

In [ ]:
weather_data['end_time'] = weather_data['start_time'].astype(int) + 100 ##add end time

In [ ]:
#now if END_TIME > 2399, subtract 2400
test = weather_data.end_time > 2399
column_name = 'end_time'
weather_data.loc[test, column_name] -= 2400


In [ ]:
weather_data['end_time'] = weather_data['end_time'].astype(str).apply(lambda x: x.zfill(4))
weather_data['start_time'] = weather_data['start_time'].astype(str).apply(lambda x: x.zfill(4))
storms['BEGIN_TIME'] = storms['BEGIN_TIME'].astype(str).apply(lambda x: x.zfill(4))

In [ ]:

weather_data['COUNTY'] = weather_data['COUNTY'].apply(str)
weather_data['STATE'] = weather_data['STATE'].apply(str)
weather_data['BEGIN_YEARMONTH'] = weather_data['BEGIN_YEARMONTH'].apply(int)
weather_data['BEGIN_DAY'] = weather_data['BEGIN_DAY'].apply(int)
weather_data['start_time'] = weather_data['start_time'].apply(int)
weather_data['end_time'] = weather_data['end_time'].apply(int)

storms['COUNTY'] = storms['COUNTY'].apply(str)
storms['STATE'] = storms['STATE'].apply(str)
storms['BEGIN_YEARMONTH'] = storms['BEGIN_YEARMONTH'].apply(int)
storms['BEGIN_DAY'] = storms['BEGIN_DAY'].apply(int)
storms['BEGIN_TIME'] = storms['BEGIN_TIME'].apply(int)


In [ ]:
#storms.loc[(storms.STATE == 'CONNECTICUT') & (storms.COUNTY == 'HARTFORD')]

In [ ]:
final = pd.merge(weather_data, storms, on=['COUNTY', 'STATE','BEGIN_YEARMONTH', 'BEGIN_DAY']) ###merge columns based on county, state, yearmonth, day, and time range (starttime is between the weather_data range)
##merge storm and weather based on county, state, month, day, year

In [ ]:
##only keep times that fall between start time and end time
final = final[(final.start_time <= final.BEGIN_TIME) & (final.BEGIN_TIME <= final.end_time)] #filter via boolean series index 

In [ ]:
#test to csv to check data
#final.to_csv(path_or_buf='testout.csv')

In [ ]:
###Remove Duplicates
final = final.drop_duplicates()

In [ ]:
final = final.drop_duplicates(subset=['EVENT_ID'])

In [ ]:
del final['STATION_NAME']
del final['LATITUDE']
del final['LONGITUDE']
del final['WBAN']
del final['DAILYCoolingDegreeDays']
del final['DAILYDeptFromNormalAverageTemp']
del final['DAILYSnowDepth']
del final['DAILYMaximumDryBulbTemp']
del final['DAILYMinimumDryBulbTemp']
del final['DAILYWeather']
del final['CATEGORY']
del final['COUNTY'], 
del final['ST']
del final['BEGIN_DATE_TIME']
del final['BEGIN_DAY']
del final['BEGIN_LAT']
del final['BEGIN_LOCATION']
del final['BEGIN_LON']
del final['BEGIN_RANGE']
del final['BEGIN_TIME']
del final['BEGIN_YEARMONTH']
del final[ 'start_time'],final[ 'STATE'],final['end_time'],final['END_YEARMONTH'], final['END_DAY'],final['END_TIME']
del final[ 'EPISODE_ID'],final[ 'STATE_FIPS'],final['YEAR'],final['MONTH_NAME'], final['EVENT_ID'],final['CZ_TYPE'], final['CZ_FIPS']
del final[ 'WFO'],final[ 'CZ_TIMEZONE'],final['END_DATE_TIME'],final['INJURIES_DIRECT'], final['INJURIES_INDIRECT'],final['DEATHS_DIRECT'], final['DEATHS_INDIRECT']
del final[ 'DAMAGE_PROPERTY'],final[ 'DAMAGE_CROPS'],final['SOURCE'],final['MAGNITUDE_TYPE'], final['FLOOD_CAUSE'],final['TOR_F_SCALE'], final['TOR_LENGTH']
del final[ 'TOR_WIDTH'],final[ 'TOR_OTHER_WFO'],final['TOR_OTHER_CZ_STATE'],final['TOR_OTHER_CZ_FIPS'], final['TOR_OTHER_CZ_NAME'],final['BEGIN_AZIMUTH'], final['END_RANGE']
del final[ 'END_AZIMUTH'],final[ 'END_LOCATION'],final['END_LAT'],final['END_LON'], final['EPISODE_NARRATIVE'],final['EVENT_NARRATIVE'], final['DATA_SOURCE']
del final['HOURLYPressureChange'], final['HOURLYPressureTendency'], final['HOURLYSeaLevelPressure']
final = final.fillna(0)
final.loc[final.HOURLYPrecip == 'T', ['HOURLYPrecip']] = 0
final.loc[final.HOURLYWindDirection == 'VRB', ['HOURLYWindDirection']] = 0

In [ ]:
labels = final['EVENT_TYPE']
del final['EVENT_TYPE']

In [ ]:
final

In [ ]:
####ALGO

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final, labels, test_size=0.25, random_state=42)

In [ ]:
##NAIVE BAYES
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
accuracy_score(y_test, y_pred)